In [1]:
import os
from PIL import Image

In [2]:
os.chdir('D:\SmartCity')

# 大图

In [8]:
datasets = os.listdir(dataset)
outpath='D:\\newSmartCity'
if not os.path.isdir(outpath):
    os.mkdir(outpath)
for i in datasets:
    pre_name = i.split('_')[0]+'_'+ i.split('_')[-1]
    sub_data=os.path.join(dataset,i)
    if not os.path.isdir(os.path.join(outpath,i.split('_')[0])):  
        os.mkdir(os.path.join(outpath,i.split('_')[0]))
    for j in os.listdir(sub_data):
        if os.path.isdir(os.path.join(sub_data,j)):
            im=Image.open(os.path.join(sub_data,j,'origin_1 Panorama.jpg'))
            newname = os.path.join(outpath,i.split('_')[0],j+'_pano.jpg')
            if not os.path.isdir(os.path.join(outpath,i.split('_')[0])):  
                os.mkdir(os.path.join(outpath,i.split('_')[0]))
            im.save(newname)

# 小图

In [8]:
dataset = 'D:\SmartCity'
datasets = os.listdir(dataset)
outpath='D:\\subSmartCity'
if not os.path.isdir(outpath):
    os.mkdir(outpath)
for i in datasets:
    pre_name = i.split('_')[0]+'_'+ i.split('_')[-1]
    sub_data=os.path.join(dataset,i)
    if not os.path.isdir(os.path.join(outpath,i.split('_')[0])):  
        os.mkdir(os.path.join(outpath,i.split('_')[0]))
    for j in os.listdir(sub_data):
        if os.path.isdir(os.path.join(sub_data,j)):
            im=Image.open(os.path.join(sub_data,j,'origin_1 Panorama.jpg'))
            im = im.resize((3788, 1509),Image.ANTIALIAS)
            newname = os.path.join(outpath,i.split('_')[0],j+'_pano.jpg')
            if not os.path.isdir(os.path.join(outpath,i.split('_')[0])):  
                os.mkdir(os.path.join(outpath,i.split('_')[0]))
            im.save(newname)

In [1]:
import json
import csv

In [2]:
import numpy
import numpy as np
import math

def quaternion_matrix(quaternion):
    # Return homogeneous rotation matrix from quaternion.
    q = numpy.array(quaternion, dtype=numpy.float64, copy=True)
    n = numpy.dot(q, q)
    if n < _EPS:
        return numpy.identity(4)
    q *= math.sqrt(2.0 / n)
    q = numpy.outer(q, q)
    return numpy.array([
        [1.0-q[2, 2]-q[3, 3],     q[1, 2]-q[3, 0],     q[1, 3]+q[2, 0], 0.0],
        [    q[1, 2]+q[3, 0], 1.0-q[1, 1]-q[3, 3],     q[2, 3]-q[1, 0], 0.0],
        [    q[1, 3]-q[2, 0],     q[2, 3]+q[1, 0], 1.0-q[1, 1]-q[2, 2], 0.0],
        [                0.0,                 0.0,                 0.0, 1.0]])

def affine_matrix_from_points(v0, v1, shear=True, scale=True, usesvd=True):
    v0 = numpy.array(v0, dtype=numpy.float64, copy=True)
    v1 = numpy.array(v1, dtype=numpy.float64, copy=True)

    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")

    # move centroids to origin
    t0 = -numpy.mean(v0, axis=1)
    M0 = numpy.identity(ndims+1)
    M0[:ndims, ndims] = t0
    v0 += t0.reshape(ndims, 1)
    t1 = -numpy.mean(v1, axis=1)
    M1 = numpy.identity(ndims+1)
    M1[:ndims, ndims] = t1
    v1 += t1.reshape(ndims, 1)

    if shear:
        # Affine transformation
        A = numpy.concatenate((v0, v1), axis=0)
        u, s, vh = numpy.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]
        C = vh[ndims:2*ndims]
        t = numpy.dot(C, numpy.linalg.pinv(B))
        t = numpy.concatenate((t, numpy.zeros((ndims, 1))), axis=1)
        M = numpy.vstack((t, ((0.0,)*ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        # Rigid transformation via SVD of covariance matrix
        u, s, vh = numpy.linalg.svd(numpy.dot(v1, v0.T))
        # rotation matrix from SVD orthonormal bases
        R = numpy.dot(u, vh)
        if numpy.linalg.det(R) < 0.0:
            # R does not constitute right handed system
            R -= numpy.outer(u[:, ndims-1], vh[ndims-1, :]*2.0)
            s[-1] *= -1.0
        # homogeneous transformation matrix
        M = numpy.identity(ndims+1)
        M[:ndims, :ndims] = R
    else:
        # Rigid transformation matrix via quaternion
        # compute symmetric matrix N
        xx, yy, zz = numpy.sum(v0 * v1, axis=1)
        xy, yz, zx = numpy.sum(v0 * numpy.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = numpy.sum(v0 * numpy.roll(v1, -2, axis=0), axis=1)
        N = [[xx+yy+zz, 0.0,      0.0,      0.0],
             [yz-zy,    xx-yy-zz, 0.0,      0.0],
             [zx-xz,    xy+yx,    yy-xx-zz, 0.0],
             [xy-yx,    zx+xz,    yz+zy,    zz-xx-yy]]
        # quaternion: eigenvector corresponding to most positive eigenvalue
        w, V = numpy.linalg.eigh(N)
        q = V[:, numpy.argmax(w)]
        q /= vector_norm(q)  # unit quaternion
        # homogeneous transformation matrix
        M = quaternion_matrix(q)

    if scale and not shear:
        # Affine transformation; scale is ratio of RMS deviations from centroid
        v0 *= v0
        v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(numpy.sum(v1) / numpy.sum(v0))

    # move centroids back
    M = numpy.dot(numpy.linalg.inv(M1), numpy.dot(M, M0))
    M /= M[ndims, ndims]
    return M

def superimposition_matrix(v0, v1, scale=False, usesvd=True):
    # Return matrix to transform given 3D point set into second point set.
    v0 = numpy.array(v0, dtype=numpy.float64, copy=False)[:3]
    v1 = numpy.array(v1, dtype=numpy.float64, copy=False)[:3]
    return affine_matrix_from_points(v0, v1, shear=False,
                                     scale=scale, usesvd=usesvd)

def align_reconstruction_naive_similarity(X, Xp):
    """Align with GPS and GCP data using direct 3D-3D matches."""
    # Compute similarity Xp = s A X + b
    T = superimposition_matrix(X.T, Xp.T, scale=True)
    A, b = T[:3, :3], T[:3, 3]
    s = np.linalg.det(A)**(1. / 3)
    A /= s
    return s, A, b


In [47]:
with open(r"./s1recon/sfm-data.json","r") as f:
    data = {}
    cc = json.loads(f.read())    
    for i in cc['extrinsics']:
        filename=cc['views'][i['key']]['value']['ptr_wrapper']['data']['filename']
        data[filename] = i['value']

        # scene3_siping_zhonghe_training
        # scene4_jiading_zhixin_training\jiading_zhixin_training_coordinates.csv
        # D:\SmartCity\scene5_jiading_rainbow_training\jiading_rainbow__training_coordinates.csv
        # D:\SmartCity\scene6_jiading_bolou_training\jiading_bolou_training_coordinates.csv
        # D:\SmartCity\scene7_jiading_riverside_training\jiading_riverside_training_coordinates
        # D:\SmartCity\scene8_jiading_hualou_training\jiading_hualou_training_coordinates
csv_file = csv.reader(open('./scene1_jiading_lib_training_coordinates.csv','r',encoding='utf-8'))

csv_data=[]
for stu in csv_file:
    csv_data.append(stu)

csv_data = csv_data[1:]

data_train={}
for i in csv_data:
    if (i[0]+'_pano.jpg') in data:
        data_train[i[0]+'.jpg'] = data[i[0]+'_pano.jpg']

In [48]:
aa=[]
bb=[]
name=[]
for i in range(len(csv_data)):
    if (csv_data[i][0]+'.jpg') in data_train:
        a=np.array(csv_data[i][1:],dtype=np.float)
        b=np.array(data_train[csv_data[i][0]+'.jpg']['center'],dtype=np.float)
        aa.append(a)
        bb.append(b)
        name.append(csv_data[i][0])

aa=np.array(aa)
bb=np.array(bb)

# s, A, b=align_reconstruction_naive_similarity(bb, aa)
# new_b=s*A.dot(bb.T).T+b

s, A, b=align_reconstruction_naive_similarity(aa, bb)
new_a=s*A.dot(aa.T).T+b


In [49]:
import copy

In [50]:
cc

{'control_points': [],
 'extrinsics': [{'key': 0,
   'value': {'center': [-0.10646129896333095,
     0.042256723018426715,
     0.40604583034421604],
    'rotation': [[0.8381960605856844,
      -0.0014877733796039069,
      -0.5453669870362622],
     [-0.027320119325113593, 0.9986262068594504, -0.04471364504899879],
     [0.5446842893811984, 0.05237829229630574, 0.8370039064409566]]}},
  {'key': 1,
   'value': {'center': [-0.10646983237491436,
     0.04225689765933745,
     0.40605591304292776],
    'rotation': [[0.9169697240781643,
      0.0008469100946722555,
      -0.3989558971456704],
     [-0.021462359246080188, 0.9986544033329707, -0.04720963714391708],
     [0.3983790811019166, 0.0518523427335705, 0.915754029362379]]}},
  {'key': 2,
   'value': {'center': [-0.10647587154917101,
     0.042255859768841206,
     0.4060366947329606],
    'rotation': [[0.9654012970208704,
      0.011531018777691242,
      -0.2605136681949138],
     [-0.023418556444764996, 0.9988188484909114, -0.04257

In [51]:
copy_json = copy.deepcopy(cc)
for i in copy_json['extrinsics']:
    if i['key'] == copy_json['views'][i['key']]['key']:
        if copy_json['views'][i['key']]['value']['ptr_wrapper']['data']['filename'][:-9] in name:
            ind = name.index(copy_json['views'][i['key']]['value']['ptr_wrapper']['data']['filename'][:-9])
            newpose = new_a[ind]
            i['value']['center'] = list(newpose)

In [54]:
def rewrite_json_file(filepath,json_data):
    with open(filepath, 'w') as f:
        json.dump(json_data,f)
    f.close()
    
rewrite_json_file('change.json',copy_json)

In [ ]:
cc=[]

data_name = data.keys()filepath
for i in data_name:
    c =np.array(data[i]['center'],dtype=np.float)
    cc.append(c)

cc=np.array(cc)

new_c=s*A.dot(cc.T).T+b


train_img = list()
for img in csv_data:
    train_img.append(img[0])

with open(r'D:\newSmartCity\results8.txt','w') as f:
    for i,img in enumerate(data_name):
        name = img.split('.')[0][:-5]
        if name not in train_img:
            f.write(name+',%.4f'%new_c[i][0]+',%.4f'%new_c[i][1]+',%.4f'%new_c[i][2]+'\n')

In [1]:
import numpy as np

with open(r'D:\newSmartCity\resultss.txt', 'r') as f:
    datas = [line.split(',') for line in f.read().splitlines()]

with open(r'D:\newSmartCity\b.txt', 'r') as f:
    target = [line.split(',') for line in f.read().splitlines()]

In [97]:
len(datas)

369

In [98]:
len(target)

373

In [2]:
with open(r'D:\newSmartCity\changxinyuan.csv', 'r') as f:
    cnn = [line.split(',') for line in f.read().splitlines()]


In [100]:
len(cnn)

373

In [3]:
ids = list()
for t in target:
    ids.append(t[0])

result = dict()
for d in datas:
    result[d[0]] = d[1:]

error = dict()
for d in cnn:
    error[d[0]] = d[1:]
    
a = list()
for t in ids:
    if t in result:
        for i in result[t]:
            t = t+','+i
    else:
        for i in error[t]:
            t = t+','+i
    a.append(t)

In [4]:
len(a)

373

In [5]:
with open(r'D:\newSmartCity\c.csv','w') as f:
    for i in a:
        f.write(i+'\n')


In [116]:
import os
import numpy as np

# union
if os.path.exists(r'D:\newSmartCity\c.txt'):
    os.remove(r'D:\newSmartCity\c.txt')

with open(r'D:\newSmartCity\c.txt','a') as f:
    for i in np.arange(1,9):
        with open(r'D:\newSmartCity\results'+str(i)+'.txt','r') as r:
            l = r.readlines()
            for j in l:
                f.write(j)

# sort
with open(r'D:\newSmartCity\c.txt', 'r') as f:
    datas = [line.split(',') for line in f.read().splitlines()]

with open(r'D:\newSmartCity\b.txt', 'r') as f:
    target = [line.split(',') for line in f.read().splitlines()]

ids = list()
for t in target:
    ids.append(t[0])

result = dict()
for d in datas:
    result[d[0]] = d[1:]

a = list()
for t in ids:
    if t in result.keys():
        for i in result[t]:
           t = t+','+i
        a.append(t)
    else:
        a.append(t)

with open(r'D:\newSmartCity\c.csv','w') as f:
    for i in a:
        f.write(i+'\n')



In [115]:
len(a)

373